In [1]:
# Minicons Installation
# Introduction can be found https://kanishka.xyz/post/minicons-running-large-scale-behavioral-analyses-on-transformer-lms/
# Tutorial and code can be found https://github.com/kanishkamisra/minicons/blob/master/examples/surprisals.md
#!pip install minicons

from minicons import scorer
import pandas as pd
import numpy as np
import json
import csv
import re
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import torch
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = 'phonemetransformers/GPT2-85M-BPE-TXT'

model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2TokenizerFast.from_pretrained(model_path)

# wrap with minicons scorer
lm_scorer = scorer.IncrementalLMScorer(model_path, device = 'cpu')

In [3]:
def calculate_surprisal(sentence):
    '''
    Takes in a sentence, and outputs surprisal values for each word.
    '''
    
    input_sentence = sentence
    # token_score() function of Minicons takes in several parameters
    # if surprisal = True, the output value is surprisal instead of log likelihood
    # if base_two = True, the log likelihood will be in base 2
    # see Minicons documentations for details
    # score tokens
    token_surprisals = lm_scorer.token_score(input_sentence, surprisal = True, base_two = True)[0]

    # filter out special tokens (like <pad>)
    special_tokens = set(tokenizer.all_special_tokens)
    filtered = [
        (token, score)
        for (token, score) in token_surprisals
        if token not in special_tokens
    ]

   # expand tokens that contain multiple words
    expanded = []
    #for token, score in token_surprisals:
    for token, score in filtered:
        token = token.strip('Ġ') # remove space marker
        token = token.lower()
        if token.count('Ġ') > 0:
            # multiple words inside
            words = token.split('Ġ')
            words = [word for word in words if word]  # remove empty strings
            for i, word in enumerate(words):
                expanded_token = word
                expanded.append((expanded_token, score / len(words)))  # split surprisal equally
        else:
            expanded.append((token, score))    
    #print(expanded)

    # use regex to split into words and punctuation
    words = re.findall(r'\w+|[^\w\s]', sentence.lower())
    results = []

    token_pointer = 0

    for word in words:
        accumulated = ''
        word_surprisal = 0.0

        while token_pointer < len(expanded):
            token, surprisal = expanded[token_pointer]
            accumulated += token
            word_surprisal += surprisal
            token_pointer += 1

            if accumulated == word:
                results.append((word, word_surprisal))
                break
        else:
            results.append((word, word_surprisal))

    return results


sentence = 'I know that your friend gave a baguette to Mary last weekend.'
calculate_surprisal(sentence)

[('i', 4.853168964385986),
 ('know', 4.774721145629883),
 ('that', 4.265200614929199),
 ('your', 6.805907249450684),
 ('friend', 5.889679908752441),
 ('gave', 8.006753921508789),
 ('a', 5.558826446533203),
 ('baguette', 22.458303689956665),
 ('to', 1.5427972078323364),
 ('mary', 11.063645362854004),
 ('last', 6.772266864776611),
 ('weekend', 9.848917961120605),
 ('.', 0.49021247029304504)]

In [4]:
def calculate_sentence_surprisal(word_surprisals):
    '''Returns total surprisal and average surprisal per word.'''
    
    scores = [score for word, score in word_surprisals]
    total = sum(scores)
    avg = total / len(scores)
    return total, avg

"""
def sum_region_surprisal(row, region_list):
    '''Sums surprisals of a specified sentence region, extracting the relevant part from sentence_surprisals using index information.'''

    sentence_surprisals = row['surprisals']
    
    # retrieve all sentence column(from 'prefix' till 'end')
    all_columns = row.index.tolist()  # get the list of column names
    prefix_index = all_columns.index('prefix')
    eos_index = all_columns.index('end')
    
    # reconstruct the full sentence from 'prefix' to 'end' (exclude NaN values)
    full_sentence = ' '.join([str(row[col]) if pd.notna(row.get(col)) else '' for col in all_columns[prefix_index:eos_index + 1]])
    
    # use regex to split the full sentence into words & punctuation
    sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)
    
    # get the start and end indices for the region_list within the full sentence
    first_region_column = region_list[0]
    first_region_start_index = all_columns.index(first_region_column)
    
    last_region_column = region_list[-1]
    last_region_end_index = all_columns.index(last_region_column)
    
    # extract the relevant slice of sentence_surprisals that corresponds to the region_list
    # calculate the starting and ending index for the region_list part
    start_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:first_region_start_index]
        if pd.notna(row[col]) 
    ])
    end_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:last_region_end_index + 1]
        if pd.notna(row[col]) 
    ])

    # extract the slice of sentence_surprisals corresponding to the region_list part
    relevant_surprisals = sentence_surprisals[start_index:end_index]
    region_surprisal = sum([score for token, score in relevant_surprisals])
    
    return region_surprisal
"""

def sum_region_surprisal(row, region_list, priority_region = None, normalize = False):
    '''Sums surprisals of a specified sentence region with optional priority region handling.'''

    sentence_surprisals = row['surprisals']
    
    # retrieve all sentence column(from 'prefix' till 'end')
    all_columns = row.index.tolist()  # get the list of column names
    prefix_index = all_columns.index('prefix')
    eos_index = all_columns.index('end')
    
    # reconstruct the full sentence from 'prefix' to 'end' (exclude NaN values)
    full_sentence = ' '.join([str(row[col]) if pd.notna(row.get(col)) else '' for col in all_columns[prefix_index:eos_index + 1]])
    
    # use regex to split the full sentence into words & punctuation
    sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)
    
    if priority_region:
        # check if priority_region has non-zero surprisal
        if pd.notna(row[priority_region]):
            # extract text for the priority_region
            priority_region_index = all_columns.index(priority_region)
            
            start_index = sum([
                len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:priority_region_index]
                if pd.notna(row[col]) 
            ])
            end_index = sum([
                len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:priority_region_index + 1]
                if pd.notna(row[col]) 
            ])
            
            # calculate the surprisal for the priority region
            relevant_surprisals = sentence_surprisals[start_index:end_index]
            priority_surprisal = sum([score for token, score in relevant_surprisals])
    
            # return priority region's surprisal score if it is non-zero
            if priority_surprisal != 0:
                return priority_surprisal
    
    # if priority region is not provided or its surprisal is zero, calculate sum of region_list surprisals

    # get the start and end indices for the region_list within the full sentence
    first_region_column = region_list[0]
    first_region_start_index = all_columns.index(first_region_column)
    
    last_region_column = region_list[-1]
    last_region_end_index = all_columns.index(last_region_column)
    
    # extract the relevant slice of sentence_surprisals that corresponds to the region_list
    # calculate the starting and ending index for the region_list part
    start_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:first_region_start_index]
        if pd.notna(row[col]) 
    ])
    end_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:last_region_end_index + 1]
        if pd.notna(row[col]) 
    ])

    # extract the slice of sentence_surprisals corresponding to the region_list part
    relevant_surprisals = sentence_surprisals[start_index:end_index]
    region_surprisal = sum([score for token, score in relevant_surprisals])
    
    if normalize == False:
        return region_surprisal
    elif normalize == True:
        return region_surprisal / len(relevant_surprisals)

<>:24: SyntaxWarning: invalid escape sequence '\w'
<>:24: SyntaxWarning: invalid escape sequence '\w'
/var/folders/vv/sc6b0f596n7dy9mbxlcnn1180000gq/T/ipykernel_13096/2872888706.py:24: SyntaxWarning: invalid escape sequence '\w'
  sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)


In [5]:
def split_ends(ends):
    '''Splits off sentence-final punctuation.'''
    
    match = re.match(r'^(.*?)([.!?])$', ends.strip())
    if match:
        return pd.Series([match.group(1), match.group(2)])
    else:
        return pd.Series([ends, ''])  # no end punctuation
    
def encode_wh_licensor(df):
    '''
    Adds a numeric column to the DataFrame:
    - 1 if licensor is a wh-licensor
    - -1 otherwise (licensor is "that" or absent)
    '''
    df['wh_numeric'] = df['filler'].apply(lambda x: 1 if x > 0 else -1)
    return df
    
def merge_sentence(row, syntactic_parts):
    '''Merges sentence columns back into full sentence.'''
    
    parts = [row[part] for part in syntactic_parts]         
    # filter out NaN or empty parts
    non_empty_parts = [str(part).strip() for part in parts if pd.notna(part) and str(part).strip() != '']
    # join with spaces and add end punctuation
    sentence = ' '.join(non_empty_parts) + (row['end'] if pd.notna(row['end']) else '')
    return sentence

def assign_grammaticality(row):
    ''' Determines grammaticality based on licensor and gap values.'''
    if row['filler'] > 0 and row['gap'] == 1:
        return 'gram+'
        #return 1
    elif row['filler'] == 0 and row['gap'] == 0:
        return 'gram+'
        #return 1
    elif row['filler'] > 0 and row['gap'] == 0:
        return 'gram-'
        #return 0
    elif row['filler'] == 0 and row['gap'] == 1:
        return 'gram-'
        #return 0

#### Statistical Analysis: Mixed-Effects Linear Regression Model

In [6]:
from IPython.display import display
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

def print_summary(result):
    '''
    Prints the descriptive header from a statsmodels MixedLMResults summary,
    and displays both the fixed-effects and random-effects components as pandas DataFrames.
    '''
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ConvergenceWarning)
        
        summary_str = str(result.summary())
        lines = summary_str.split('\n')

        # PART 1: HEADER
        print("\n=== MODEL SUMMARY ===")
        for line in lines:
            if 'Coef.' in line:
                break  # stop when reaching the coefficient table
            pairs = re.findall(r'(\S[^:]*):\s+([^\s][^:]*?)(?=\s{2,}|$)', line)
            for key, value in pairs:
                print(f"{key.strip()}: {value.strip()}")

        # PART 2a: FIXED EFFECTS TABLE
        fixed_table = []
        in_fixed_table = False
        random_start_idx = None

        for i, line in enumerate(lines):
            if 'Coef.' in line:
                in_fixed_table = True
                continue
            if in_fixed_table:
                if not line.strip() or set(line.strip()) in [{'='}, {'-'}]:
                    continue  # skip empty/separator lines
                parts = line.strip().split()
                if len(parts) >= 7:
                    param = parts[0]
                    row = parts[1:7]
                    fixed_table.append([param] + row)
                else:
                    random_start_idx = i
                    break

        if fixed_table:
            df_fixed = pd.DataFrame(fixed_table, columns = [
                'Parameter', 'Coef.', 'Std.Err.', 'z', 'P>|z|', '[0.025', '0.975]'
            ])
            print("\n=== FIXED EFFECTS COEFFICIENTS ===")
            display(df_fixed)
        else:
            print("\nNo fixed-effects data found.")

        # PART 2b: RANDOM EFFECTS / VARIANCE COMPONENTS TABLE
        random_table = []
        if random_start_idx:
            for line in lines[random_start_idx:]:
                if not line.strip() or set(line.strip()) in [{'='}, {'-'}]:
                    continue
                parts = line.strip().split()
                # Look for last token ending in Var or Cov
                varcov_idx = None
                for j in reversed(range(len(parts))):
                    if parts[j].endswith('Var') or parts[j].endswith('Cov'):
                        varcov_idx = j
                        break
                if varcov_idx is not None:
                    param = ' '.join(parts[:varcov_idx + 1])
                    numeric_parts = parts[varcov_idx + 1:]
                    if len(numeric_parts) == 2:
                        val1, val2 = numeric_parts
                    elif len(numeric_parts) == 1:
                        val1, val2 = numeric_parts[0], ''
                    else:
                        val1, val2 = '', ''
                    random_table.append([param, val1, val2])

        if random_table:
            df_random = pd.DataFrame(random_table, columns = ['Parameter', 'Coef.', 'Std.Err.'])
            print("\n=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===")
            display(df_random)
        else:
            print("\nNo random-effects data found.")


In [7]:
# columns: item_id (indicates sentence set), wh_licensor (0/1), gap (0/1), island_type, surprisal

warnings.simplefilter("ignore", ConvergenceWarning)

def mixed_effects_linear_regression(df, iv_list, surprisal, label):
    '''
    Fits mixed-effects model and extracts wh-licensing interaction.
    '''
        
    for field in iv_list:
        if field == 'wh_numeric' or 'gap_numeric' or 'gap_distance_length':
            pass
        else:
            df[field] = df[field].astype('category')

    interaction_terms = ' * '.join(iv_list)
    random_effects = ' + '.join(iv_list)

    model = smf.mixedlm(
        f"{surprisal} ~ {interaction_terms}",
        df,
        groups = df["item_id"],
        #re_formula = f"~{random_effects}"
        re_formula = "1"
        )

    result = model.fit()
    #interaction_coef = result.params.get('wh_licensor[T.1]:gap[T.1]', None)

    print(f"\n=== {label.upper()} ===")
    #print(result.summary())
    print_summary(result)

    return result

#interaction = mixed_effects_linear_regression(df, "construction_type") # label name to be changed according to construction type

#### Gap Distance - Categorical/Continuous

In [13]:
sentence_df = pd.read_csv('test_sentences/Gap Distance.csv')
sentence_df['gap_distance'] = sentence_df['gap_distance'].fillna('null')
#sentence_df['gap_distance'].dropna(inplace = True)
#sentence_df = sentence_df[sentence_df['gap_distance'].replace(['nan', 'NaN'], np.nan).notna()]

FileNotFoundError: [Errno 2] No such file or directory: 'test_sentences/Gap Distance.csv'

In [ ]:
sentence_df[['temp_mod', 'end']] = sentence_df['temp_mod'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'licensor', 'subj', 'modifier', 'verb', 'object', 'prep', 'goal', 'temp_mod']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['gap_distance_length'] = sentence_df['modifier'].apply(lambda x: len(x) if pd.notna(x) else 0)
sentence_df['grammaticality'] = sentence_df.apply(assign_grammaticality, axis = 1)

Gap Position == Object

In [ ]:
# modifier: short_mod, med_mod, long_mod

In [ ]:
object_df = sentence_df.copy()
#object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['prep', 'goal']), axis = 1)
object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['prep', 'goal'], 'object'), axis = 1)

# filter for object gap
objectgap_df = object_df[object_df['gap_position'] == 'DO']

In [ ]:
# flip test: gap-
no_gap_df = objectgap_df[objectgap_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'object gap position: gap-')


=== OBJECT GAP POSITION: GAP- ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: region_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 32.9620
Min. group size: 8
Log-Likelihood: -525.5890
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,20.699,1.313,15.759,0.000,18.125,23.273
1,wh_numeric,1.211,0.454,2.668,0.008,0.321,2.101



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,30.382,2.079


In [ ]:
# flip test: gap+
gap_df = objectgap_df[objectgap_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'object gap position: gap+')


=== OBJECT GAP POSITION: GAP+ ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: region_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 39.7509
Min. group size: 8
Log-Likelihood: -533.5621
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,18.002,1.007,17.872,0.000,16.028,19.976
1,wh_numeric,-0.811,0.498,-1.627,0.104,-1.788,0.166



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,15.323,1.113


Gap Postion == PP/Goal

In [ ]:
pp_df = sentence_df.copy()
#pp_df['region_surprisal'] = pp_df.apply(lambda x: sum_region_surprisal(x, ['temp_mod']), axis = 1)
pp_df['region_surprisal'] = pp_df.apply(lambda x: sum_region_surprisal(x, ['temp_mod'], 'goal'), axis = 1)

# filter for PP gap
ppgap_df = pp_df[pp_df['gap_position'] == 'IO']

In [ ]:
# flip test: gap-
no_gap_df = ppgap_df[ppgap_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'pp gap position: gap-')


=== PP GAP POSITION: GAP- ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: region_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 24.6121
Min. group size: 8
Log-Likelihood: -501.3133
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,12.799,1.066,12.011,0.000,10.710,14.887
1,wh_numeric,0.252,0.392,0.643,0.520,-0.516,1.021



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,19.632,1.583


In [ ]:
# flip test: gap+
gap_df = ppgap_df[ppgap_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'pp gap position: gap+')


=== PP GAP POSITION: GAP+ ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: region_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 52.8580
Min. group size: 8
Log-Likelihood: -561.4271
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,23.479,1.539,15.252,0.000,20.461,26.496
1,wh_numeric,-0.680,0.575,-1.182,0.237,-1.806,0.447



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,40.789,2.255


Embedded Clause

In [ ]:
embed = ['subj', 'modifier', 'verb', 'object', 'prep', 'goal', 'temp_mod', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

Embedded Clause - Object Gap Position

In [ ]:
# filter for object gap
embed_objectgap_df = sentence_df[sentence_df['gap_position'] == 'DO']

In [ ]:
# flip test: gap-
no_gap_df = embed_objectgap_df[embed_objectgap_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'object gap position: gap-')


=== OBJECT GAP POSITION: GAP- ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.3251
Min. group size: 8
Log-Likelihood: -162.3289
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.633,0.142,53.686,0.000,7.355,7.912
1,wh_numeric,0.198,0.045,4.393,0.000,0.110,0.286



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.364,0.245


In [ ]:
# flip test: gap+
gap_df = embed_objectgap_df[embed_objectgap_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'object gap position: gap+')


=== OBJECT GAP POSITION: GAP+ ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.4312
Min. group size: 8
Log-Likelihood: -182.7351
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.757,0.148,52.371,0.000,7.466,8.047
1,wh_numeric,0.011,0.052,0.215,0.830,-0.091,0.113



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.385,0.231


In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(embed_objectgap_df, iv_list, 'embed_surprisal', 'object gap position: division by grammaticality')


=== OBJECT GAP POSITION: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.3948
Min. group size: 16
Log-Likelihood: -334.8603
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.602,0.145,52.502,0.000,7.318,7.885
1,grammaticality[T.gram-],0.187,0.070,2.660,0.008,0.049,0.325



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.370,0.210


Embedded Clause - PP/Goal Gap Position

In [ ]:
# filter for PP gap
embed_ppgap_df = sentence_df[sentence_df['gap_position'] == 'IO']

In [ ]:
# flip test: gap-
no_gap_df = embed_ppgap_df[embed_ppgap_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'pp gap position: gap-')


=== PP GAP POSITION: GAP- ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.3393
Min. group size: 8
Log-Likelihood: -166.3738
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.564,0.150,50.269,0.000,7.269,7.859
1,wh_numeric,0.128,0.046,2.786,0.005,0.038,0.219



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.410,0.269


In [ ]:
# flip test: gap+
gap_df = embed_ppgap_df[embed_ppgap_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'pp gap position: gap+')


=== PP GAP POSITION: GAP+ ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.3355
Min. group size: 8
Log-Likelihood: -162.9713
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.049,0.131,61.404,0.000,7.792,8.306
1,wh_numeric,0.059,0.046,1.295,0.195,-0.030,0.149



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.302,0.205


In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(embed_ppgap_df, iv_list, 'embed_surprisal', 'pp gap position: division by grammaticality')


=== PP GAP POSITION: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.4129
Min. group size: 16
Log-Likelihood: -340.9851
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.772,0.141,55.117,0.000,7.495,8.048
1,grammaticality[T.gram-],0.069,0.072,0.960,0.337,-0.072,0.210



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.346,0.194


#### Double Gaps

In [ ]:
sentence_df = pd.read_csv('test_sentences/Double Gaps.csv')

In [ ]:
def assign_grammaticality_double_gaps(row):
    ''' Determines grammaticality based on licensor and gap values for the double gap construction.'''
    if row['filler'] == 1 and row['gap_count'] == 1:
        return 'gram+'
    elif row['filler'] == 0 and row['gap_count'] == 0:
        return 'gram+'
    elif row['filler'] == 1 and row['gap_count'] == 0:
        return 'gram-'
    elif row['filler'] == 0 and row['gap_count'] == 1:
        return 'gram-'
    elif row['filler'] == 0 and row['gap_count'] == 2:
        return 'gram-'
    elif row['filler'] == 1 and row['gap_count'] == 2:
        return 'gram-'

In [ ]:
sentence_df[['modifier', 'end']] = sentence_df['modifier'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'licensor', 'subj', 'verb', 'object', 'modifier']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['gap_count'] = sentence_df['subject_gap'] + sentence_df['object_gap']
sentence_df['grammaticality'] = sentence_df.apply(assign_grammaticality_double_gaps, axis = 1)

Embedded Clause

In [ ]:
embed = ['subj', 'object', 'verb', 'modifier', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [ ]:
# flip test: gap = 0
gap_0 = sentence_df[sentence_df['gap_count'] == 0]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_0, iv_list, 'embed_surprisal', 'double gap embed: gap = 0')


=== DOUBLE GAP EMBED: GAP = 0 ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 40
Method: REML
No. Groups: 20
Scale: 0.0975
Min. group size: 2
Log-Likelihood: -40.1536
Max. group size: 2
Converged: Yes
Mean group size: 2.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.516,0.202,32.244,0.000,6.120,6.912
1,wh_numeric,0.439,0.049,8.893,0.000,0.342,0.536



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.768,1.200


In [ ]:
# flip test: gap = 1
gap_1 = sentence_df[sentence_df['gap_count'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_1, iv_list, 'embed_surprisal', 'double gap embed: gap = 1')


=== DOUBLE GAP EMBED: GAP = 1 ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 80
Method: REML
No. Groups: 20
Scale: 0.3054
Min. group size: 4
Log-Likelihood: -90.9827
Max. group size: 4
Converged: Yes
Mean group size: 4.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.202,0.199,36.268,0.000,6.813,7.591
1,wh_numeric,-0.067,0.062,-1.082,0.279,-0.188,0.054



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.712,0.532


In [ ]:
# flip test: gap = 2
gap_2 = sentence_df[sentence_df['gap_count'] == 2]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_2, iv_list, 'embed_surprisal', 'double gap embed: gap = 2')


=== DOUBLE GAP EMBED: GAP = 2 ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 40
Method: REML
No. Groups: 20
Scale: 0.1374
Min. group size: 2
Log-Likelihood: -50.4927
Max. group size: 2
Converged: Yes
Mean group size: 2.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.751,0.293,29.832,0.000,8.176,9.326
1,wh_numeric,-0.189,0.059,-3.223,0.001,-0.304,-0.074



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.652,2.130


In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'double gap embed: division by grammaticality')


=== DOUBLE GAP EMBED: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.9288
Min. group size: 8
Log-Likelihood: -241.5204
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.782,0.227,29.821,0.000,6.336,7.228
1,grammaticality[T.gram-],1.017,0.157,6.459,0.000,0.708,1.325



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.725,0.302


### Islands

In [ ]:
def assign_grammaticality_islands(row, island_type):
    ''' Determines grammaticality based on licensor and gap values.'''
    if row[island_type] == 'null' or row[island_type] == 'that':
        if row['filler'] > 0 and row['gap'] == 1:
            return 'gram+'
            #return 1
        elif row['filler'] == 0 and row['gap'] == 0:
            return 'gram+'
            #return 1
        elif row['filler'] > 0 and row['gap'] == 0:
            return 'gram-'
            #return 0
        elif row['filler'] == 0 and row['gap'] == 1:
            return 'gram-'
            #return 0
    else:
        if row['filler'] > 0 and row['gap'] == 1:
            return 'gram-'
            #return 0
        elif row['filler'] == 0 and row['gap'] == 0:
            return 'gram+'
            #return 1
        elif row['filler'] > 0 and row['gap'] == 0:
            return 'gram-'
            #return 0
        elif row['filler'] == 0 and row['gap'] == 1:
            return 'gram-'
            #return 0

#### Wh-Islands

In [ ]:
sentence_df = pd.read_csv('test_sentences/Wh-Islands.csv')
sentence_df['comp_type'] = sentence_df['comp_type'].fillna('null')

In [ ]:
sentence_df[['cont', 'end']] = sentence_df['cont'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'filler_word', 'subj1', 'verb1', 'comp', 'subj2', 'embed_verb', 'obj', 'cont']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence (x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['comp_type'] = pd.Categorical(sentence_df['comp_type'], categories = ['null', 'that', 'whether'], ordered = True)
sentence_df['grammaticality'] = sentence_df.apply(lambda x: assign_grammaticality_islands(x, 'comp_type'), axis = 1)

Post-Gap Region - Continuation

In [ ]:
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['cont'], 'obj'), axis = 1)

In [ ]:
# flip test: gap-
no_gap_df = sentence_df[sentence_df['gap_numeric'] == -1]
iv_list = ['wh_numeric', 'comp_type']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'wh-islands continuation: gap-')


=== WH-ISLANDS CONTINUATION: GAP- ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 1.5514
Min. group size: 6
Log-Likelihood: -243.2035
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,15.682,1.247,12.574,0.000,13.237,18.126
1,comp_type[T.that],-1.549,0.279,-5.561,0.000,-2.095,-1.003
2,comp_type[T.whether],-2.204,0.279,-7.913,0.000,-2.750,-1.658
3,wh_numeric,1.083,0.197,5.500,0.000,0.697,1.469
4,wh_numeric:comp_type[T.that],-0.501,0.279,-1.801,0.072,-1.047,0.044
5,wh_numeric:comp_type[T.whether],-1.134,0.279,-4.071,0.000,-1.680,-0.588



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,30.334,8.728


In [ ]:
# flip test: gap+
gap_df = sentence_df[sentence_df['gap_numeric'] == 1]
iv_list = ['wh_numeric', 'comp_type']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'wh-islands continuation: gap+')


=== WH-ISLANDS CONTINUATION: GAP+ ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 1.2801
Min. group size: 6
Log-Likelihood: -242.0839
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,21.848,1.891,11.555,0.000,18.142,25.554
1,comp_type[T.that],1.145,0.253,4.526,0.000,0.649,1.641
2,comp_type[T.whether],2.429,0.253,9.599,0.000,1.933,2.924
3,wh_numeric,-0.630,0.179,-3.523,0.000,-0.981,-0.280
4,wh_numeric:comp_type[T.that],-0.314,0.253,-1.243,0.214,-0.810,0.181
5,wh_numeric:comp_type[T.whether],0.589,0.253,2.329,0.020,0.093,1.085



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,70.859,22.325


Embedded Clause

In [ ]:
embed = ['subj1', 'verb1', 'comp', 'subj2', 'embed_verb', 'obj', 'cont', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [ ]:
# flip test: gap-
no_gap_df = sentence_df[sentence_df['gap_numeric'] == -1]
iv_list = ['wh_numeric', 'comp_type']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'wh-islands embed: gap-')


=== WH-ISLANDS EMBED: GAP- ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.0844
Min. group size: 6
Log-Likelihood: -76.2692
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.536,0.276,30.877,0.000,7.994,9.078
1,comp_type[T.that],-0.889,0.065,-13.683,0.000,-1.016,-0.762
2,comp_type[T.whether],-0.254,0.065,-3.912,0.000,-0.381,-0.127
3,wh_numeric,0.202,0.046,4.397,0.000,0.112,0.292
4,wh_numeric:comp_type[T.that],-0.024,0.065,-0.373,0.709,-0.152,0.103
5,wh_numeric:comp_type[T.whether],-0.239,0.065,-3.686,0.000,-0.367,-0.112



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.486,1.835


In [ ]:
# flip test: gap+
gap_df = sentence_df[sentence_df['gap_numeric'] == 1]
iv_list = ['wh_numeric', 'comp_type']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'wh-islands embed: gap+')


=== WH-ISLANDS EMBED: GAP+ ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.1213
Min. group size: 6
Log-Likelihood: -94.0664
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,9.157,0.286,32.066,0.000,8.598,9.717
1,comp_type[T.that],-0.785,0.078,-10.079,0.000,-0.938,-0.632
2,comp_type[T.whether],0.192,0.078,2.468,0.014,0.040,0.345
3,wh_numeric,0.041,0.055,0.745,0.456,-0.067,0.149
4,wh_numeric:comp_type[T.that],-0.015,0.078,-0.195,0.845,-0.168,0.137
5,wh_numeric:comp_type[T.whether],-0.116,0.078,-1.491,0.136,-0.269,0.036



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.570,1.623


In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'wh-islands embed: division by grammaticality')


=== WH-ISLANDS EMBED: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 0.4501
Min. group size: 12
Log-Likelihood: -282.6300
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.344,0.280,29.785,0.000,7.795,8.893
1,grammaticality[T.gram-],0.366,0.088,4.168,0.000,0.194,0.538



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.479,0.765


#### Adjunct Islands 

In [ ]:
sentence_df = pd.read_csv('test_sentences/Adjunct Islands.csv')
sentence_df['adjunct_type'] = sentence_df['adjunct_type'].fillna('null')

In [ ]:
sentence_df[['cont', 'end']] = sentence_df['cont'].apply(split_ends)
sentence_df['adjunct_front_trigger'] = np.nan
sentence_df['adjunct_front_trigger'] = sentence_df['adjunct_trigger'].where(sentence_df['adjunct_type'] == 'front', sentence_df['adjunct_front_trigger'])
sentence_df['adjunct_trigger'] = sentence_df['adjunct_trigger'].where(sentence_df['adjunct_type'] != 'front', np.nan)
sentence_df.rename(columns = {'adjunct_trigger': 'adjunct_back_trigger'}, inplace = True)
sentence_df = sentence_df[[col for col in sentence_df.columns if col != 'end'] + ['end']]

In [ ]:
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'filler_word', 'adjunct_back_trigger', 'prep', 'subj', 'mod', 'verb', 'obj', 'cont', 'adjunct_front_trigger']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['adjunct_type'] = pd.Categorical(sentence_df['adjunct_type'], categories = ['null', 'back', 'front'], ordered = True)
sentence_df['grammaticality'] = sentence_df.apply(lambda x: assign_grammaticality_islands(x, 'adjunct_type'), axis = 1)

Post-Gap Region - Continuation

In [ ]:
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['cont'], 'obj'), axis = 1)

In [ ]:
# flip test: gap-
no_gap_df = sentence_df[sentence_df['gap_numeric'] == -1]
iv_list = ['wh_numeric', 'adjunct_type']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'adjunct islands continuation: gap-')


=== ADJUNCT ISLANDS CONTINUATION: GAP- ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 1.4465
Min. group size: 6
Log-Likelihood: -245.2462
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,20.799,1.648,12.622,0.000,17.569,24.029
1,adjunct_type[T.back],-3.246,0.269,-12.070,0.000,-3.773,-2.719
2,adjunct_type[T.front],-2.195,0.269,-8.160,0.000,-2.722,-1.667
3,wh_numeric,1.706,0.190,8.971,0.000,1.333,2.079
4,wh_numeric:adjunct_type[T.back],-1.691,0.269,-6.289,0.000,-2.219,-1.164
5,wh_numeric:adjunct_type[T.front],-1.909,0.269,-7.099,0.000,-2.436,-1.382



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,53.581,15.904


In [ ]:
# flip test: gap+
gap_df = sentence_df[sentence_df['gap_numeric'] == 1]
iv_list = ['wh_numeric', 'adjunct_type']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'adjunct islands continuation: gap+')


=== ADJUNCT ISLANDS CONTINUATION: GAP+ ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 2.3688
Min. group size: 6
Log-Likelihood: -268.8374
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,30.992,1.666,18.598,0.000,27.726,34.258
1,adjunct_type[T.back],1.145,0.344,3.326,0.001,0.470,1.819
2,adjunct_type[T.front],1.246,0.344,3.620,0.000,0.571,1.920
3,wh_numeric,-0.950,0.243,-3.906,0.000,-1.427,-0.473
4,wh_numeric:adjunct_type[T.back],0.898,0.344,2.610,0.009,0.224,1.573
5,wh_numeric:adjunct_type[T.front],0.990,0.344,2.878,0.004,0.316,1.665



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,54.353,12.642


Embedded Clause

In [ ]:
embed = ['adjunct_back_trigger', 'prep', 'subj', 'mod', 'verb', 'obj', 'cont', 'adjunct_front_trigger', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [ ]:
# flip test: gap-
no_gap_df = sentence_df[sentence_df['gap_numeric'] == -1]
iv_list = ['wh_numeric', 'adjunct_type']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'adjunct islands embed: gap-')


=== ADJUNCT ISLANDS EMBED: GAP- ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.0903
Min. group size: 6
Log-Likelihood: -75.1424
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.096,0.221,32.084,0.000,6.663,7.530
1,adjunct_type[T.back],0.099,0.067,1.471,0.141,-0.033,0.231
2,adjunct_type[T.front],0.459,0.067,6.824,0.000,0.327,0.590
3,wh_numeric,0.360,0.048,7.582,0.000,0.267,0.453
4,wh_numeric:adjunct_type[T.back],-0.142,0.067,-2.118,0.034,-0.274,-0.011
5,wh_numeric:adjunct_type[T.front],-0.122,0.067,-1.818,0.069,-0.254,0.010



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.933,1.121


In [ ]:
# flip test: gap+
gap_df = sentence_df[sentence_df['gap_numeric'] == 1]
iv_list = ['wh_numeric', 'adjunct_type']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'adjunct islands embed: gap+')


=== ADJUNCT ISLANDS EMBED: GAP+ ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.1158
Min. group size: 6
Log-Likelihood: -86.9742
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.264,0.223,32.631,0.000,6.827,7.700
1,adjunct_type[T.back],0.353,0.076,4.638,0.000,0.204,0.502
2,adjunct_type[T.front],0.623,0.076,8.193,0.000,0.474,0.773
3,wh_numeric,0.145,0.054,2.703,0.007,0.040,0.251
4,wh_numeric:adjunct_type[T.back],0.103,0.076,1.352,0.176,-0.046,0.252
5,wh_numeric:adjunct_type[T.front],0.115,0.076,1.517,0.129,-0.034,0.265



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.933,0.995


In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'adjunct islands embed: division by grammaticality')


=== ADJUNCT ISLANDS EMBED: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 0.2461
Min. group size: 12
Log-Likelihood: -211.4051
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.110,0.216,32.934,0.000,6.687,7.533
1,grammaticality[T.gram-],0.489,0.068,7.195,0.000,0.356,0.622



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.871,0.608


#### Complex NP Islands

In [ ]:
sentence_df = pd.read_csv('test_sentences/Complex NP Islands.csv')

In [ ]:
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
sentence_df['sentence'] = sentence_df.apply(merge_sentence, axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['grammaticality'] = sentence_df.apply(assign_grammaticality_islands, axis = 1)

Subject Condition

In [ ]:
subject_df = sentence_df.copy()

In [ ]:
subject_df['region_surprisal'] = subject_df.apply(lambda x: sum_region_surprisal(x, ['subj_setup']), axis = 1)
subject_df

In [ ]:
# filter for subject condition
subject_df = subject_df[subject_df['subj_obj'] == 'subject']

In [ ]:
# flip test: gap-
no_gap_df = subject_df[subject_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'complex np islands subject condition: gap-')

In [ ]:
# flip test: gap+
gap_df = subject_df[subject_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'complex np islands subject condition: gap+')

In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(subject_df, iv_list, 'region_surprisal', 'complex np islands subject condition: division by grammaticality')

Object Condition

In [ ]:
object_df = sentence_df.copy()

In [ ]:
object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['end']), axis = 1)
object_df

In [ ]:
# filter for object gap
object_df = object_df[object_df['subj_obj'] == 'object']

In [ ]:
# flip test: gap-
no_gap_df = object_df[object_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'complex np islands object condition: gap-')

In [ ]:
# flip test: gap+
gap_df = object_df[object_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'complex np islands object condition: gap+')

In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(object_df, iv_list, 'region_surprisal', 'complex np islands object condition: division by grammaticality')

Embedded Clause

In [ ]:
embed = ['subj', 'that_rc', 'what_rc', 'rc_np', 'prep', 'prep_np', 'subj_setup', 'obj_setup', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)
sentence_df

Embedded Clause - Subject Condition

In [ ]:
# filter for subject condition
subject_df = sentence_df[sentence_df['subj_obj'] == 'subject']

In [ ]:
# flip test: gap-
no_gap_df = subject_df[subject_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'complex np islands subject condition: gap-')

In [ ]:
# flip test: gap+
gap_df = subject_df[subject_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'complex np islands subject condition: gap+')

In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(subject_df, iv_list, 'embed_surprisal', 'complex np islands subject condition: division by grammaticality')

Embedded Clause - Object Condition

In [ ]:
# filter for object gap
object_df = sentence_df[sentence_df['subj_obj'] == 'object']

In [ ]:
# flip test: gap-
no_gap_df = object_df[object_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'complex np islands object condition: gap-')

In [ ]:
# flip test: gap+
gap_df = object_df[object_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'complex np islands object condition: gap+')

In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(object_df, iv_list, 'embed_surprisal', 'complex np islands object condition: division by grammaticality')